In [1]:
import gzip
from pathlib import Path
from io import BufferedReader, BytesIO
from teleparser.decoders.ber import BerDecoder
from teleparser.file_handling import BufferManager

folder = Path.cwd().parent / "data"

In [2]:
file = folder / "timvoz.gz"
buffer_manager = BufferManager(file)

In [3]:
ber = BerDecoder()
file_buffer = buffer_manager.open_file()


In [4]:
if (tlv := ber.decode_tlv(file_buffer)) is not None:
    print("Varredura: (Call Data Record)")
    print(tlv.tag.string, tlv.tag.tag_class, tlv.tag.number, tlv.length, tlv.value)
    if tlv.children is not None:
        for child in tlv.children:
            print(80*'=')
            print("Call Module")
            print(f"tipo={child.tag.number}, comprimento={child.length}, bytes={child.value}")
            print(80 * "=")
            print(f'Quantidade de parâmetros dentro do Call Module: {len(child.children)}')
            print("Parâmetros:")
            for c in child.children:
                print(
                    f"tipo={c.tag.number}, comprimento={c.length}, bytes={c.value}"
                )


Varredura: (Call Data Record)
a0 BerClass.CONTEXT 0 191 b'\xa2\x81\xbc\x99\x01\x01\x8c\x04\x00\x00\x00\x00\x89\x03\x18\x0c\x0e\x93\x0fZCTA09P BR00308\x8d\x03\x00\x00\x00\x82\x03\x16\x08Z\x91\x02\x00\x01\x8a\x03\x0b\x00\x0f\x8b\x03\x0b\x00\x0f\x9f0\x02%?\x85\x07A@\x93\x88\x11q\x08\x87\r\x11U\n#\x01\x00\x004\x89\x18\x11\x87\xf0\x94\x07\x11U\x00D\x01\x01\x90\x86\x08\x01#\x01\x00\x00\x00\x00\xf0\x9f1\x05-\xa0 %?\x84\x07A4)\x00R!\xf0\x96\x0741TWYMI\x9f,\x010\x9e\x01\x00\x9f)\x03|;\xac\x81\x03|;\xe2\x83\x01\x01\x80\x03\x00\x02\x04\x98\x01\x00\x90\x01\x00\x9b\x02\x02\x17\x8e\x03\x00\x00\x00\x8f\x01\x00\x88\x01\x02\x95\x07ZFN3AEO\x9a\x016'
Call Module
tipo=2, comprimento=188, bytes=b'\x99\x01\x01\x8c\x04\x00\x00\x00\x00\x89\x03\x18\x0c\x0e\x93\x0fZCTA09P BR00308\x8d\x03\x00\x00\x00\x82\x03\x16\x08Z\x91\x02\x00\x01\x8a\x03\x0b\x00\x0f\x8b\x03\x0b\x00\x0f\x9f0\x02%?\x85\x07A@\x93\x88\x11q\x08\x87\r\x11U\n#\x01\x00\x004\x89\x18\x11\x87\xf0\x94\x07\x11U\x00D\x01\x01\x90\x86\x08\x01#\x01\x00\x00\x0

In [ ]:
%%time
ber = BerDecoder()
with buffer_manager.open() as file_buffer:
    counter = 0
    while (tlv := ber.decode_tlv(file_buffer)) is not None:
        counter += 1

print(f"Numbers of CDR blocks: {counter}")